In [45]:
%matplotlib inline

import sys
import warnings
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import LabelEncoder

import hyperopt
import xgboost as xgb

warnings.filterwarnings('ignore')

In [34]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not list(actual):
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [35]:
data_train = pd.read_csv('/opt/devs/pershin/kf/train.csv')
data_train.drop('row_id', axis=1, inplace=True)

print('Train data size: {:0.01f} Mb'.format(data_train.memory_usage().sum() / 1024**2))

Train data size: 1110.8 Mb


In [6]:
data_test = pd.read_csv('/opt/devs/pershin/kf/test.csv')
data_test.drop('row_id', axis=1, inplace=True)

print('Test data size: {:0.01f} Mb'.format(data_test.memory_usage().sum() / 1024**2))

Data size: 262.7 Mb


## Additional functions

In [41]:
def filter_rare_places(data: pd.DataFrame, min_n=10):
    place_counts = data.place_id.value_counts()
    mask = place_counts[data.place_id.values] >= min_n
    return data.loc[mask.values]

def filter_rare_classes(X, y, min_n=10, return_indices=False):
    _class, _class_count = np.unique(y, return_counts=True)
    _rare_place_ids_indices = np.zeros_like(y).astype(bool)
    for rare_place_id in _class[_class_count < min_n]:
        _rare_place_ids_indices[y == rare_place_id] = True
    if return_indices:
        return X[~_rare_place_ids_indices,:], y[~_rare_place_ids_indices], (~_rare_place_ids_indices)
    else:
        return X[~_rare_place_ids_indices,:], y[~_rare_place_ids_indices]

In [12]:
def get_max_n_indices(array: np.array, n: int):
    return array.argsort()[::-1][:n]

def get_n_most_frequent(array: np.array, n: int):
    vals, counts = np.unique(array, return_counts=True)
    return vals[counts.argsort()[::-1][:n]]

def replicate(array: np.array):
    return np.concatenate([[val]*(len(array) - i) for i,val in enumerate(array)])

In [13]:
def train_test_split_by_time(data, train_size=78):
    time_validate_threshold = np.percentile(data.time.values, train_size)  # as train data ~22% of train+test data

    train_indices = data.time.values < time_validate_threshold
    train_place_ids = np.unique(data.place_id.iloc[train_indices].values)
    
    test_indices = (data.time.values >= time_validate_threshold) & np.in1d(data.place_id.values, train_place_ids)
    test_place_ids = np.unique(data.place_id.iloc[test_indices].values)
    
    return data.iloc[train_indices, :], data.iloc[test_indices, :]

# Feature engineering

In [15]:
def add_features(data: pd.DataFrame):
    data['hour'] = (data.time.values / 60) % 24  # keep the minutes info
    data['weekday'] = ((data.time.values // 60) / 24) % 7
    data['day'] = ((data.time.values // 60) / 24) % 30
    data['month'] = ((data.time.values // (60 * 24)) / 30) % 12
    data['year'] = data.time.values // (60 * 24 * 365)
    data['x_d_y'] = data.x.values / (data.y.values + 1e-5)
    data['x_t_y'] = data.x.values * data.y.values
    
    accuracy_through_time = data.sort_values(by='time')[['time', 'accuracy']].groupby('time').agg(np.median)
    median_accuracy_through_days = pd.rolling_median(accuracy_through_time, window=60*24)
    data['accuracy_diff'] = data.accuracy.values - median_accuracy_through_days.loc[data.time].accuracy.values
    data['accuracy_diff'][np.isnan(data['accuracy_diff'])] = 0
    
    data['log10_accuracy'] = np.log10(data.accuracy.values)

    return data

In [13]:
data_train = add_features(data_train)
data_test = add_features(data_test)

In [53]:
# data_cv = data_train.query('0.25 < x < 0.5 and 0.25 < y < 0.5')
data_cv = data_train.query('0 < x < 1 and 0 < y < 1')

In [54]:
data_cv = add_features(data_cv)

In [26]:
features_to_use = [
    'x', 'y', 'x_d_y', 'x_t_y',
    'hour', 'weekday', 'month', 'year',
    'accuracy', 'accuracy_diff'
]

In [57]:
kde_features_to_use = ['x', 'y', 'log10_accuracy', 'hour', 'weekday', 'time']

# Models

In [56]:
class GridEstimator:
    def __init__(self, estimator, bandwidths_for_kde_features, features_probas_weights):
        self.estimator = estimator
        self.bandwidths_for_kde_features = bandwidths_for_kde_features
        self.features_probas_weights = features_probas_weights
        
    def query(self, train_data: pd.DataFrame, test_data: pd.DataFrame,
              feature_names, kde_features_names, grid_size, epsilan, min_place_id_occurences=0):
        x_cells = grid_size[0]
        y_cells = grid_size[1]
        x_range = np.linspace(train_data.x.min(), train_data.x.max(), num=x_cells+1)
        y_range = np.linspace(train_data.y.min(), train_data.y.max(), num=y_cells+1)
        x_range[-1] = np.inf
        y_range[-1] = np.inf

        margin_x = ((train_data.x.max() - train_data.x.min()) / x_cells) * epsilan[0]
        margin_y = ((train_data.y.max() - train_data.y.min()) / y_cells) * epsilan[1]
        
        places_features_kdes = {}
        for _feature in self.bandwidths_for_kde_features:
            places_features_kdes[_feature] = filter_rare_places(train_data).groupby('place_id').apply(
                lambda x: KernelDensity(
                    kernel='gaussian',
                    metric='manhattan',
                    bandwidth=self.bandwidths_for_kde_features[_feature]
                ).fit(
                    np.vstack([
                        x[_feature].values[:, np.newaxis] - features_periods[_feature],
                        x[_feature].values[:, np.newaxis],
                        x[_feature].values[:, np.newaxis] + features_periods[_feature]
                    ]) if features_periods[_feature] else x[_feature].values[:, np.newaxis]
                )
            )   
  
        
        predictions = np.zeros((len(test_data), 3))

        for x_min, x_max in zip(x_range[:-1], x_range[1:]):
            for y_min, y_max in zip(y_range[:-1], y_range[1:]):

                _train_idx = \
                    ((x_min - margin_x) <= train_data.x.values) & \
                    (train_data.x.values < (x_max + margin_x)) & \
                    ((y_min - margin_y) <= train_data.y.values) & \
                    (train_data.y.values < (y_max + margin_y))
                
                _test_idx = \
                    ((x_min) <= test_data.x.values) & \
                    (test_data.x.values < (x_max)) & \
                    ((y_min) <= test_data.y.values) & \
                    (test_data.y.values < (y_max))


                _train_data = filter_rare_places(train_data[_train_idx], min_n=min_place_id_occurences)
                _test_data  = test_data[_test_idx]
    
                _X_train = _train_data[feature_names].values            
                _X_test = _test_data[feature_names].values
            
                le = LabelEncoder()
                _y_train = le.fit_transform(_train_data.place_id.values)

                places_kde_probas  = []
                for _feature in kde_features:

                    _places_kde_probas  = np.empty((len(_X_test),  len(le.classes_)))

                    for _j, _place_id in enumerate(le.classes_):
                        _places_kde_probas[:,_j] = np.exp(
                            places_features_kdes[_feature][_place_id].score_samples(
                                _test_data[_feature].values[:, np.newaxis]
                            )
                        )

                    if self.features_probas_weights:                            
                        _places_kde_probas = _places_kde_probas.mean() + \
                            (_places_kde_probas - _places_kde_probas.mean()) * \
                            self.features_probas_weights[_feature]
                    
                    places_kde_probas.append(_places_kde_probas)

                places_kde_probas = np.prod(places_kde_probas, axis=0)


                self.estimator.fit(_X_train, _y_train)
                
                _pred_probas = self.estimator.predict_proba(_X_test)
                _pred_probas = _pred_probas * places_kde_probas

                _y_pred = _pred_probas.argsort(axis=1)[:,::-1][:,:3]
                predictions[_test_idx, :] = le.inverse_transform(_y_pred)
        
        return predictions.astype(int)

# CV:

##### These are some hyperparameters that have been estimated with hyperopt

In [39]:
bandwidths_for_kde_features = {
    'x': 1e-2,
    'y': 5e-3,
    'log10_accuracy': 2e-1,
    'time': 1e5,
    'hour': 2e0,
    'weekday':6e-1,
    'day': 4e-1,
    'month': 1e0
}

features_periods = {
    'x': 0,
    'y': 0,
    'log10_accuracy': 0,
    'time': 0,
    'hour': 24,
    'weekday': 7,
    'day': 30,
    'month': 12
}

features_probas_weights = {
    'hour': 0.957,
    'log10_accuracy': 0.909,
    'time': 0.721,
    'weekday': 0.225,
    'x': 0.525,
    'y': 0.985
}

In [55]:
start_time = time.time()

model = GridEstimator(
    xgb.XGBClassifier(
        max_depth=5,
        n_estimators=100,
        learning_rate=0.15,
        objective='multi:softprob',
        nthread=4,
        subsample=0.525,
        colsample_bytree=0.9,
        seed=0
    ),
    bandwidths_for_kde_features=bandwidths_for_kde_features,
    features_probas_weights=features_probas_weights
)

cv_train_data, cv_validate_data = train_test_split_by_time(data_cv)

cv_y_valid = cv_validate_data.place_id.values.reshape(-1,1)
cv_y_pred = model.query(
    cv_train_data,
    cv_validate_data,
    features_to_use,
    kde_features_to_use,
    grid_size=(8, 12),
    epsilan=(0.7, 0.1),
    min_place_id_occurences=10
)

cv_score = mapk(cv_y_valid, cv_y_pred, k=3)

end_time = time.time()
print('{:0.04f}'.format(cv_score))
print('time spend: {:0.01f}s'.format(end_time - start_time))

0.6009
time spend: 834.1s


## Predict and save

In [17]:
model = GridEstimator(
    xgb.XGBClassifier(
        max_depth=5,
        n_estimators=100,
        learning_rate=0.15,
        objective='multi:softprob',
        nthread=4,
        subsample=0.525,
        colsample_bytree=0.9,
        seed=0
    ),
    bandwidths_for_kde_features=bandwidths_for_kde_features,
    features_probas_weights=features_probas_weights
)

In [ ]:
start_time = time.time()

y_pred = model.query(
    data_train,
    data_test,
    features_to_use,
    kde_features_to_use,
    grid_size=(80, 120),
    epsilan=(0.7, 0.1),
    min_place_id_occurences=10
)

end_time = time.time()
print('Completed! Time spend: {:0.01f}s'.format(end_time - start_time))

In [ ]:
with open('data/y_pred.csv', 'w') as f:
    print('row_id,place_id', file=f)
    for row, prediction in enumerate(y_pred):
        print('{},{}'.format(row, ' '.join(prediction.astype(int).astype(str))), file=f)